<a href="https://colab.research.google.com/github/vennelasri4111/Fmml/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [2]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [3]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [4]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [5]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [6]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [7]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [8]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [9]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [10]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [11]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [12]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [13]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [14]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [15]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
Answer: Averaging validation accuracy across multiple splits generally leads to more consistent and reliable results. Here's why:

1. **Mitigation of Variability**: Each data split can introduce variability in model performance due to differences in the training and validation sets. By averaging across multiple splits, you smooth out these variations, reducing the impact of any single split that may be unusually good or bad.

2. **Generalization**: Averaging across splits helps you better estimate how well your model will generalize to unseen data. This approach is particularly useful in situations where your dataset might be small or not fully representative.

3. **Reduced Overfitting Risk**: When you evaluate your model on just one split, there's a risk of overfitting to that particular validation set. Averaging over multiple splits helps mitigate this by ensuring your model isn't overly tuned to the peculiarities of one specific split.

4. **More Robust Evaluation**: Averaging allows you to account for the inherent randomness in the training process, such as the initial weights or the order of data presentation during training, leading to a more robust evaluation metric.

However, it's important to note that while averaging across splits gives more consistent results, the choice of the number of splits (e.g., in cross-validation) and ensuring that each split is done correctly are also crucial for obtaining meaningful results.
2. Does it give more accurate estimate of test accuracy?
Answer:Yes, averaging validation accuracy across multiple splits generally provides a more accurate estimate of test accuracy. Here’s why:

1. **Reduced Bias**: When you average across multiple splits, you reduce the risk of your validation accuracy being biased by the peculiarities of a single train-test split. This makes the average validation accuracy a better reflection of how the model might perform on unseen data.

2. **Better Generalization**: By evaluating your model on different portions of the data in each split, you get a more comprehensive assessment of its performance. This helps in estimating how well the model will generalize to the test set, which often contains data distributions slightly different from those seen in any individual validation split.

3. **Mitigation of Overfitting**: Overfitting to a single validation set can lead to an overly optimistic estimate of test accuracy. Averaging across multiple splits helps to counteract this by providing a more balanced view, less influenced by any one particular split.

4. **Variance Reduction**: Different splits may produce different validation accuracies due to random factors like the distribution of data points in each split. Averaging these values reduces the variance of your estimate, making it more reliable.

5. **Cross-Validation Principle**: In k-fold cross-validation, the model is trained and validated k times, each on different splits of the data. The average of these k validation accuracies is often used as an estimate of test accuracy, and research has shown that this average typically aligns closely with the actual test accuracy, assuming the test data is drawn from the same distribution as the training/validation data.

Therefore, averaging validation accuracy across multiple splits does provide a more accurate and reliable estimate of the model’s performance on the test set.
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
Answer:Increasing the number of iterations, or the number of splits (as in cross-validation), generally improves the estimate of test accuracy, but there are some nuances to consider:

### 1. **Variance Reduction**:
   - **Lower Iterations**: When you use a small number of splits or iterations, the estimate of test accuracy can be more variable. Each split might give you a slightly different accuracy due to the random distribution of data, which can lead to a less stable estimate.
   - **Higher Iterations**: As you increase the number of iterations, the effect of randomness in the splits diminishes. This leads to a more stable and reliable estimate because you're averaging over more diverse samples of the data.

### 2. **Bias-Variance Trade-off**:
   - **Bias**: The average of many iterations (or splits) tends to reduce bias. If you use too few iterations, you might get an estimate that is biased toward the specific data used in those splits.
   - **Variance**: More iterations tend to lower the variance of the estimate, making it less sensitive to the random fluctuations in any single split. This produces a more consistent and dependable estimate of test accuracy.

### 3. **Computational Cost**:
   - **Lower Iterations**: Fewer iterations require less computational power and time, making them more efficient. However, this might come at the cost of a less accurate estimate.
   - **Higher Iterations**: More iterations improve the accuracy of the estimate but at the expense of increased computational resources and time. The law of diminishing returns applies here—beyond a certain point, additional iterations yield only marginal improvements in the estimate.

### 4. **Convergence**:
   - In practice, as the number of iterations increases, the estimate of test accuracy converges to a stable value. Once this convergence is reached, additional iterations add little value in terms of improving the estimate.

### 5. **Practical Considerations**:
   - In many cases, a 5- or 10-fold cross-validation is sufficient to get a reliable estimate of test accuracy. If your data is particularly noisy or small, increasing the number of iterations might be necessary to get a better estimate.

### Conclusion:
Yes, increasing the number of iterations typically provides a better estimate of test accuracy by reducing variance and giving a more robust measure of model performance. However, there is a balance to be struck between the number of iterations and the computational resources available. Beyond a certain point, the improvements gained from additional iterations may be minimal, so it’s important to consider the trade-offs.
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?
Answer:Increasing the number of iterations (e.g., through more splits in cross-validation) can help mitigate some of the issues associated with a very small training or validation dataset, but it doesn't entirely solve the problem. Here's why:

### 1. **Benefits of Increasing Iterations**:
   - **Variance Reduction**: With a very small dataset, the model's performance can vary greatly depending on the specific data points in the training and validation sets. By increasing the number of iterations, you average these results, which reduces the variance and gives a more stable estimate of performance.
   - **Better Utilization of Data**: In methods like k-fold cross-validation, each data point gets used for both training and validation across different iterations. This ensures that even a small dataset is fully utilized, giving you a more complete understanding of the model’s performance.

### 2. **Limitations with Very Small Datasets**:
   - **High Variability**: Even with increased iterations, a very small dataset can lead to high variability in model performance across splits, as the training data in each iteration might not be representative of the underlying data distribution.
   - **Overfitting**: With very small datasets, there’s a high risk of overfitting. The model might perform well on the training data but fail to generalize to new data, even if you average over many iterations.
   - **Bias in Estimation**: Even with many iterations, the model might still have a high bias if the small dataset doesn’t capture the full complexity of the problem. Averaging more results doesn’t address the fundamental issue of the dataset being too limited to train a robust model.

### 3. **Diminishing Returns**:
   - **Limited Gains**: While increasing iterations can give you a more stable estimate, the gains become marginal after a certain point, especially with very small datasets. The underlying problem of insufficient data remains, limiting how much you can trust the results.

### 4. **Practical Considerations**:
   - **Data Augmentation**: Instead of (or alongside) increasing iterations, techniques like data augmentation can be used to artificially increase the size of the training dataset. This can sometimes provide a more effective solution than just increasing the number of iterations.
   - **Ensemble Methods**: Using ensemble techniques that combine the predictions of multiple models trained on different splits or subsets can also help improve performance with small datasets.

### Conclusion:
Increasing the number of iterations can improve the reliability of your performance estimates when dealing with a small dataset, but it’s not a complete solution. The fundamental limitations of a very small dataset—such as overfitting, high variance, and potentially high bias—remain significant challenges. To effectively deal with very small datasets, you may need to combine this approach with other strategies, such as data augmentation, transfer learning, or ensemble methods.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.